# [ 4-3. 현금흐름 계산3 ]
## 1. 필요한 모듈 import
* 기존 작성한 모듈 및 필요 파일 확인
    - FCHotel_FSmodeling_assumption.xlsx
    - m00_general_function.py
    - m01_assumption.py
    - m02_index.py
    - m03_funding.py
    - m04_operating_income.py
    - m05_operating_cost.py
    - m06_facility_cost.py

* 앞에서 작성한 general_function의 df_seperator 함수도 추가

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 20)
pd.set_option('display.width', 300)

# DataFrame의 출력을 확장하여 한 줄로 계속 출력되도록 설정
pd.set_option('display.expand_frame_repr', True)

# 앞에서 작성한 general_function의 df_seperator 함수도 추가
from m00_general_function import df_seperator

from m01_assumption import assumption
from m02_index import index
from m03_funding import funding
from m04_operating_income import operating_income
from m05_operating_cost import operating_cost
from m06_facility_cost import facility_cost

## 2. 운영비용 현금흐름 작성
### 2-1. operating_cost 객체 확인

In [2]:
operating_cost.keys()

dict_keys(['객실운영비', '관리운영비', '인건비'])

In [3]:
operating_cost['객실운영비']

{'청소세탁비':                TypeA     TypeB     TypeC     Total
 2023-12-31         0         0         0         0
 2024-01-31  11160000  13392000  10540000  35092000
 2024-02-29   9860000  11832000   9280000  30972000
 2024-03-31   9920000  11904000   8680000  30504000
 2024-04-30   9600000  11520000   8400000  29520000
 ...              ...       ...       ...       ...
 2026-09-30  10184640  12221568   9548100  31954308
 2026-10-31  10524128  12628953   9866370  33019451
 2026-11-30   8911560  10693872   8911560  28516992
 2026-12-31  11839644  14207572  11839644  37886860
 2027-01-31         0         0         0         0
 
 [38 rows x 4 columns],
 '수도광열비':               TypeA    TypeB    TypeC  Overhead     Total
 2023-12-31        0        0        0   3000000   3000000
 2024-01-31  5580000  6696000  5270000   3000000  20546000
 2024-02-29  4930000  5916000  4640000   3000000  18486000
 2024-03-31  4960000  5952000  4340000   3000000  18252000
 2024-04-30  4800000  5760000  420000

### 2-2. operating_cost - 객실운영비 객체의 현금흐름 반영

In [4]:
#### cashflow, balance 객체 초기화 ####
cashflow = pd.DataFrame(
    columns = ['date', 'categoryA', 'categoryB', 'categoryC', '입금금액', '출금금액']
)

balance = pd.DataFrame({
    '기초현금': [0] * len(index['model']),
    '입금금액': [0] * len(index['model']),
    '출금금액': [0] * len(index['model']),
    '기말현금': [0] * len(index['model']), 
},
    index=index['model']
)
room_type_list = list(assumption['business_overview']['객실수'].keys())


#### cashflow, balance 작성 ####
idx = 0
cash_balance = 0
for dt in index['model']:
    #### 0. 기초현금 계산
    balance.loc[dt, '기초현금'] = cash_balance

    #### 1. 자금조달소요
    ## 1-1. 자기자본 유입
    amount = funding['자기자본'].loc[dt, '자기자본유입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자금조달', '자기자본', '자기자본유입', amount, 0]
        balance.loc[dt, '입금금액'] += amount
        idx += 1

    ## 1-2. 차입금 유입
    amount = funding['차입금'].loc[dt, '차입금유입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자금조달', '차입금', '차입금유입', amount, 0]
        balance.loc[dt, '입금금액'] += amount
        idx += 1

    ## 1-3. 자산매입
    amount = funding['자산매입'].loc[dt, '자산매입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자산매입', '자산매입', '매입대금지출', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = funding['자산매입'].loc[dt, '매입부수비용']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자산매입', '매입부수비용', '부수비용지출', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    #### 2. 운영수입
    for room_type in room_type_list:
        amount = operating_income[room_type].loc[dt, '객실수입']
        if amount > 0:
            cashflow.loc[idx] = [dt, '운영수입', '객실수입', room_type, amount, 0]
            balance.loc[dt, '입금금액'] += amount
            idx += 1

    #### 3. 운영비용 - 객실운영비
    ## 3-1. 청소세탁비
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['청소세탁비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '청소세탁비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1
    
    ## 3-2. 수도광열비
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['수도광열비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '수도광열비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1

    amount = operating_cost['객실운영비']['수도광열비'].loc[dt, 'Overhead']
    if amount > 0:
        cashflow.loc[idx] = [dt, '객실운영비', '수도광열비', 'Overhead', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    
    ## 3-3. 예약수수료
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['예약수수료'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '예약수수료', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1


    #### 9. 기말현금 계산
    cash_balance = (
        balance.loc[dt, '기초현금'] + balance.loc[dt, '입금금액'] - balance.loc[dt, '출금금액']
    )
    balance.loc[dt, '기말현금'] = cash_balance

In [5]:
cashflow

,date,categoryA,categoryB,categoryC,입금금액,출금금액
0,2023-12-31,자금조달,자기자본,자기자본유입,10000000000,0
1,2023-12-31,자금조달,차입금,차입금유입,10000000000,0
2,2023-12-31,자산매입,자산매입,매입대금지출,0,18000000000
3,2023-12-31,자산매입,매입부수비용,부수비용지출,0,1000000000
4,2023-12-31,객실운영비,수도광열비,Overhead,0,3000000
...,...,...,...,...,...,...
457,2026-12-31,객실운영비,수도광열비,Overhead,0,3182700
458,2026-12-31,객실운영비,예약수수료,TypeA,0,4352400
459,2026-12-31,객실운영비,예약수수료,TypeB,0,5022000
460,2026-12-31,객실운영비,예약수수료,TypeC,0,4185000


In [6]:
balance

,기초현금,입금금액,출금금액,기말현금
2023-12-31,0,20000000000,19003000000,997000000
2024-01-31,997000000,444230000,68964900,1372265100
2024-02-29,1372265100,392080000,61220400,1703124700
2024-03-31,1703124700,305040000,57907200,1950257500
2024-04-30,1950257500,295200000,56136000,2189321500
...,...,...,...,...
2026-09-30,10109043677,301200000,60150162,10350093515
2026-10-31,10350093515,311240000,62049076,10599284439
2026-11-30,10599284439,268800000,54022188,10814062251
2026-12-31,10814062251,451980000,73572390,11192469861


### 2-3. operating_cost - 관리운영비 객체의 현금흐름 반영

In [7]:
#### cashflow, balance 객체 초기화 ####
cashflow = pd.DataFrame(
    columns = ['date', 'categoryA', 'categoryB', 'categoryC', '입금금액', '출금금액']
)

balance = pd.DataFrame({
    '기초현금': [0] * len(index['model']),
    '입금금액': [0] * len(index['model']),
    '출금금액': [0] * len(index['model']),
    '기말현금': [0] * len(index['model']), 
},
    index=index['model']
)
room_type_list = list(assumption['business_overview']['객실수'].keys())


#### cashflow, balance 작성 ####
idx = 0
cash_balance = 0
for dt in index['model']:
    #### 0. 기초현금 계산
    balance.loc[dt, '기초현금'] = cash_balance

    #### 1. 자금조달소요
    ## 1-1. 자기자본 유입
    amount = funding['자기자본'].loc[dt, '자기자본유입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자금조달', '자기자본', '자기자본유입', amount, 0]
        balance.loc[dt, '입금금액'] += amount
        idx += 1

    ## 1-2. 차입금 유입
    amount = funding['차입금'].loc[dt, '차입금유입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자금조달', '차입금', '차입금유입', amount, 0]
        balance.loc[dt, '입금금액'] += amount
        idx += 1

    ## 1-3. 자산매입
    amount = funding['자산매입'].loc[dt, '자산매입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자산매입', '자산매입', '매입대금지출', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = funding['자산매입'].loc[dt, '매입부수비용']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자산매입', '매입부수비용', '부수비용지출', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    #### 2. 운영수입
    for room_type in room_type_list:
        amount = operating_income[room_type].loc[dt, '객실수입']
        if amount > 0:
            cashflow.loc[idx] = [dt, '운영수입', '객실수입', room_type, amount, 0]
            balance.loc[dt, '입금금액'] += amount
            idx += 1

    #### 3. 운영비용 - 객실운영비
    ## 3-1. 청소세탁비
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['청소세탁비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '청소세탁비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1
    
    ## 3-2. 수도광열비
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['수도광열비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '수도광열비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1

    amount = operating_cost['객실운영비']['수도광열비'].loc[dt, 'Overhead']
    if amount > 0:
        cashflow.loc[idx] = [dt, '객실운영비', '수도광열비', 'Overhead', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    
    ## 3-3. 예약수수료
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['예약수수료'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '예약수수료', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1
    
    #### 4. 운영비용 - 관리운영비
    ## 4-1. 광고홍보비
    amount = operating_cost['관리운영비']['관리운영비'].loc[dt, '광고홍보비']
    if amount > 0:
        cashflow.loc[idx] = [dt, '관리운영비', '관리운영비', '광고홍보비', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    
    ## 4-2. 기타운영비
    amount = operating_cost['관리운영비']['관리운영비'].loc[dt, '기타운영비']
    if amount > 0:
        cashflow.loc[idx] = [dt, '관리운영비', '관리운영비', '기타운영비', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1


    #### 9. 기말현금 계산
    cash_balance = (
        balance.loc[dt, '기초현금'] + balance.loc[dt, '입금금액'] - balance.loc[dt, '출금금액']
    )
    balance.loc[dt, '기말현금'] = cash_balance

In [10]:
cashflow[cashflow['categoryA'] == '관리운영비']

,date,categoryA,categoryB,categoryC,입금금액,출금금액
5,2023-12-31,관리운영비,관리운영비,광고홍보비,0,30000000
6,2023-12-31,관리운영비,관리운영비,기타운영비,0,10000000
20,2024-01-31,관리운영비,관리운영비,광고홍보비,0,30000000
21,2024-01-31,관리운영비,관리운영비,기타운영비,0,10000000
35,2024-02-29,관리운영비,관리운영비,광고홍보비,0,30000000
...,...,...,...,...,...,...
519,2026-11-30,관리운영비,관리운영비,기타운영비,0,10609000
533,2026-12-31,관리운영비,관리운영비,광고홍보비,0,31827000
534,2026-12-31,관리운영비,관리운영비,기타운영비,0,10609000
536,2027-01-31,관리운영비,관리운영비,광고홍보비,0,32781810


### 2-4. operating_cost - 인건비 객체의 현금흐름 반영

In [11]:
#### cashflow, balance 객체 초기화 ####
cashflow = pd.DataFrame(
    columns = ['date', 'categoryA', 'categoryB', 'categoryC', '입금금액', '출금금액']
)

balance = pd.DataFrame({
    '기초현금': [0] * len(index['model']),
    '입금금액': [0] * len(index['model']),
    '출금금액': [0] * len(index['model']),
    '기말현금': [0] * len(index['model']), 
},
    index=index['model']
)
room_type_list = list(assumption['business_overview']['객실수'].keys())


#### cashflow, balance 작성 ####
idx = 0
cash_balance = 0
for dt in index['model']:
    #### 0. 기초현금 계산
    balance.loc[dt, '기초현금'] = cash_balance

    #### 1. 자금조달소요
    ## 1-1. 자기자본 유입
    amount = funding['자기자본'].loc[dt, '자기자본유입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자금조달', '자기자본', '자기자본유입', amount, 0]
        balance.loc[dt, '입금금액'] += amount
        idx += 1

    ## 1-2. 차입금 유입
    amount = funding['차입금'].loc[dt, '차입금유입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자금조달', '차입금', '차입금유입', amount, 0]
        balance.loc[dt, '입금금액'] += amount
        idx += 1

    ## 1-3. 자산매입
    amount = funding['자산매입'].loc[dt, '자산매입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자산매입', '자산매입', '매입대금지출', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = funding['자산매입'].loc[dt, '매입부수비용']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자산매입', '매입부수비용', '부수비용지출', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    #### 2. 운영수입
    for room_type in room_type_list:
        amount = operating_income[room_type].loc[dt, '객실수입']
        if amount > 0:
            cashflow.loc[idx] = [dt, '운영수입', '객실수입', room_type, amount, 0]
            balance.loc[dt, '입금금액'] += amount
            idx += 1

    #### 3. 운영비용 - 객실운영비
    ## 3-1. 청소세탁비
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['청소세탁비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '청소세탁비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1
    
    ## 3-2. 수도광열비
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['수도광열비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '수도광열비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1

    amount = operating_cost['객실운영비']['수도광열비'].loc[dt, 'Overhead']
    if amount > 0:
        cashflow.loc[idx] = [dt, '객실운영비', '수도광열비', 'Overhead', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    
    ## 3-3. 예약수수료
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['예약수수료'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '예약수수료', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1
    
    #### 4. 운영비용 - 관리운영비
    ## 4-1. 광고홍보비
    amount = operating_cost['관리운영비']['관리운영비'].loc[dt, '광고홍보비']
    if amount > 0:
        cashflow.loc[idx] = [dt, '관리운영비', '관리운영비', '광고홍보비', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    
    ## 4-2. 기타운영비
    amount = operating_cost['관리운영비']['관리운영비'].loc[dt, '기타운영비']
    if amount > 0:
        cashflow.loc[idx] = [dt, '관리운영비', '관리운영비', '기타운영비', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    #### 5. 운영비용 - 인건비
    ## 5-1. 객실운영팀
    amount = operating_cost['인건비']['객실운영팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '객실운영팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = operating_cost['인건비']['객실운영팀'].loc[dt, '임시직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '객실운영팀', '임시직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    ## 5-2. 경영지원팀
    amount = operating_cost['인건비']['경영지원팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '경영지원팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = operating_cost['인건비']['경영지원팀'].loc[dt, '임원']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '경영지원팀', '임원', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    ## 5-3. 마케팅팀
    amount = operating_cost['인건비']['마케팅팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '마케팅팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    ## 5-4. 시설관리팀
    amount = operating_cost['인건비']['시설관리팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '시설관리팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = operating_cost['인건비']['시설관리팀'].loc[dt, '임시직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '시설관리팀', '임시직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1


    #### 9. 기말현금 계산
    cash_balance = (
        balance.loc[dt, '기초현금'] + balance.loc[dt, '입금금액'] - balance.loc[dt, '출금금액']
    )
    balance.loc[dt, '기말현금'] = cash_balance

In [12]:
cashflow[cashflow['categoryA'] == '인건비']

,date,categoryA,categoryB,categoryC,입금금액,출금금액
7,2023-12-31,인건비,객실운영팀,정규직,0,15287671
8,2023-12-31,인건비,객실운영팀,임시직,0,5095890
9,2023-12-31,인건비,경영지원팀,정규직,0,9172602
10,2023-12-31,인건비,경영지원팀,임원,0,6794520
11,2023-12-31,인건비,마케팅팀,정규직,0,6115068
...,...,...,...,...,...,...
799,2027-01-31,인건비,경영지원팀,정규직,0,10618434
800,2027-01-31,인건비,경영지원팀,임원,0,7865506
801,2027-01-31,인건비,마케팅팀,정규직,0,7078956
802,2027-01-31,인건비,시설관리팀,정규직,0,7078956


## 3. 시설관리비 현금흐름 작성
### 3-1. facility_cost 객체 확인

In [13]:
facility_cost.keys()

dict_keys(['통상수선비', '대수선공사비', 'TypeA', 'TypeB', 'TypeC', 'Total'])

In [14]:
facility_cost['통상수선비']

,TypeA,TypeB,TypeC,Total
2023-12-31,0,0,0,0
2024-01-31,1116000,1339200,1054000,3509200
2024-02-29,986000,1183200,928000,3097200
2024-03-31,992000,1190400,868000,3050400
2024-04-30,960000,1152000,840000,2952000
...,...,...,...,...
2026-09-30,1018464,1222156,954810,3195430
2026-10-31,1052412,1262895,986637,3301944
2026-11-30,891156,1069387,891156,2851699
2026-12-31,1183964,1420757,1183964,3788685


### 3-2. facility_cost 객체의 현금흐름 반영

In [15]:
#### cashflow, balance 객체 초기화 ####
cashflow = pd.DataFrame(
    columns = ['date', 'categoryA', 'categoryB', 'categoryC', '입금금액', '출금금액']
)

balance = pd.DataFrame({
    '기초현금': [0] * len(index['model']),
    '입금금액': [0] * len(index['model']),
    '출금금액': [0] * len(index['model']),
    '기말현금': [0] * len(index['model']), 
},
    index=index['model']
)
room_type_list = list(assumption['business_overview']['객실수'].keys())


#### cashflow, balance 작성 ####
idx = 0
cash_balance = 0
for dt in index['model']:
    #### 0. 기초현금 계산
    balance.loc[dt, '기초현금'] = cash_balance

    #### 1. 자금조달소요
    ## 1-1. 자기자본 유입
    amount = funding['자기자본'].loc[dt, '자기자본유입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자금조달', '자기자본', '자기자본유입', amount, 0]
        balance.loc[dt, '입금금액'] += amount
        idx += 1

    ## 1-2. 차입금 유입
    amount = funding['차입금'].loc[dt, '차입금유입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자금조달', '차입금', '차입금유입', amount, 0]
        balance.loc[dt, '입금금액'] += amount
        idx += 1

    ## 1-3. 자산매입
    amount = funding['자산매입'].loc[dt, '자산매입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자산매입', '자산매입', '매입대금지출', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = funding['자산매입'].loc[dt, '매입부수비용']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자산매입', '매입부수비용', '부수비용지출', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    #### 2. 운영수입
    for room_type in room_type_list:
        amount = operating_income[room_type].loc[dt, '객실수입']
        if amount > 0:
            cashflow.loc[idx] = [dt, '운영수입', '객실수입', room_type, amount, 0]
            balance.loc[dt, '입금금액'] += amount
            idx += 1

    #### 3. 운영비용 - 객실운영비
    ## 3-1. 청소세탁비
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['청소세탁비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '청소세탁비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1
    
    ## 3-2. 수도광열비
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['수도광열비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '수도광열비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1

    amount = operating_cost['객실운영비']['수도광열비'].loc[dt, 'Overhead']
    if amount > 0:
        cashflow.loc[idx] = [dt, '객실운영비', '수도광열비', 'Overhead', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    
    ## 3-3. 예약수수료
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['예약수수료'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '예약수수료', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1
    
    #### 4. 운영비용 - 관리운영비
    ## 4-1. 광고홍보비
    amount = operating_cost['관리운영비']['관리운영비'].loc[dt, '광고홍보비']
    if amount > 0:
        cashflow.loc[idx] = [dt, '관리운영비', '관리운영비', '광고홍보비', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    
    ## 4-2. 기타운영비
    amount = operating_cost['관리운영비']['관리운영비'].loc[dt, '기타운영비']
    if amount > 0:
        cashflow.loc[idx] = [dt, '관리운영비', '관리운영비', '기타운영비', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    #### 5. 운영비용 - 인건비
    ## 5-1. 객실운영팀
    amount = operating_cost['인건비']['객실운영팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '객실운영팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = operating_cost['인건비']['객실운영팀'].loc[dt, '임시직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '객실운영팀', '임시직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    ## 5-2. 경영지원팀
    amount = operating_cost['인건비']['경영지원팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '경영지원팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = operating_cost['인건비']['경영지원팀'].loc[dt, '임원']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '경영지원팀', '임원', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    ## 5-3. 마케팅팀
    amount = operating_cost['인건비']['마케팅팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '마케팅팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    ## 5-4. 시설관리팀
    amount = operating_cost['인건비']['시설관리팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '시설관리팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = operating_cost['인건비']['시설관리팀'].loc[dt, '임시직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '시설관리팀', '임시직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    #### 6. 시설관리비
    ## 6-1. 통상수선비
    for room_type in room_type_list:
        amount = facility_cost['통상수선비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '시설관리비', '통상수선비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1

    ## 6-2. 대수선공사비
    for room_type in room_type_list:
        amount = facility_cost['대수선공사비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '시설관리비', '대수선공사비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1


    #### 9. 기말현금 계산
    cash_balance = (
        balance.loc[dt, '기초현금'] + balance.loc[dt, '입금금액'] - balance.loc[dt, '출금금액']
    )
    balance.loc[dt, '기말현금'] = cash_balance

In [17]:
cashflow[cashflow['categoryA'] == '시설관리비']

,date,categoryA,categoryB,categoryC,입금금액,출금금액
36,2024-01-31,시설관리비,통상수선비,TypeA,0,1116000
37,2024-01-31,시설관리비,통상수선비,TypeB,0,1339200
38,2024-01-31,시설관리비,통상수선비,TypeC,0,1054000
61,2024-02-29,시설관리비,통상수선비,TypeA,0,986000
62,2024-02-29,시설관리비,통상수선비,TypeB,0,1183200
...,...,...,...,...,...,...
875,2026-11-30,시설관리비,통상수선비,TypeB,0,1069387
876,2026-11-30,시설관리비,통상수선비,TypeC,0,891156
899,2026-12-31,시설관리비,통상수선비,TypeA,0,1183964
900,2026-12-31,시설관리비,통상수선비,TypeB,0,1420757
